<a href="https://colab.research.google.com/github/djliden/numerai/blob/main/notebooks/regressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introduction
This notebook will walk you through the entire process of making a [numerai](numer.ai) submission, from downloading the data to submitting final predictions, all in a google colab notebook. In particular, it will address two challenges:
- handling API keys in a remote environment (colab)
- parsing the large CSV files which, if read all at once, will exceed colab's memory and cause the notebook to crash.

This notebook will implement two models: a basic tabular neural network using `fastai` and a linear regression model using `scikit-learn`.

## 1.1 Installing and Importing Dependencies
First, we install and import the necessary packages. This cell is currently set *not* to print any output; if you run into any issues and need to check for error messages, comment out the `%%capture` line

In [1]:
%%capture
# install
!pip install --upgrade python-dotenv numerapi

# import dependencies
import gc
import os
from dotenv import load_dotenv, find_dotenv
from getpass import getpass
import pandas as pd
import numpy as np
import numerapi
from pathlib import Path
from scipy.stats import spearmanr
import sklearn.linear_model
from tqdm import tqdm

## 1.2 Setting Up numerapi
We will use the [numerapi](https://github.com/uuazed/numerapi) package to access the data and make submissions. For this to work, numerapi needs to use your API keys (which can be obtained [here](https://numer.ai/submit)). We will set up two main ways of passing these API keys to a numerapi instance:
1. Read a `.env` file using the `python-dotenv` package. This will require you to upload a `.env` file (which contains your secret key and should *not* be kept under version control). Using this method means you will not have to directly enter your keys each time you use this notebook, though you will need to re-upload the `.env` file.
2. Manually entering the API keys -- if you don't have access to, or don't want to mess with, your `.env` file.

If you have a `.env` file, upload it to the default working directory, `content`, now. In either case, run the cell below to set up the numerapi instance. See [Appendix A](#app_a) for instructions on generating and downloading a .env file.

In [2]:
# Load the numerapi credentials from .env or prompt for them if not available
def credential():
    dotenv_path = find_dotenv()
    load_dotenv(dotenv_path)

    if os.getenv("NUMERAI_PUBLIC_KEY"):
        print("Loaded Numerai Public Key into Global Environment!")
    else:
        os.environ["NUMERAI_PUBLIC_KEY"] = getpass("Please enter your Numerai Public Key. You can find your key here: https://numer.ai/submit -> ")
    
    if os.getenv("NUMERAI_SECRET_KEY"):
        print("Loaded Numerai Secret Key into Global Environment!")
    else:
        os.environ["NUMERAI_SECRET_KEY"] = getpass("Please enter your Numerai Secret Key. You can find your key here: https://numer.ai/submit -> ")
    
    if os.getenv("NUMERAI_MODEL_ID_REGRESSIONS"):
        print("Loaded Numerai Model ID into Global Environment!")
    else:
        os.environ["NUMERAI_MODEL_ID_REGRESSIONS"] = getpass("Please enter your Numerai Model ID. You can find your key here: https://numer.ai/submit -> ")

credential()
public_key = os.environ.get("NUMERAI_PUBLIC_KEY")
secret_key = os.environ.get("NUMERAI_SECRET_KEY")
model_id = os.environ.get("NUMERAI_MODEL_ID_REGRESSIONS")
napi = numerapi.NumerAPI(verbosity="info", public_id=public_key, secret_key=secret_key)

Loaded Numerai Public Key into Global Environment!
Loaded Numerai Secret Key into Global Environment!
Loaded Numerai Model ID into Global Environment!


You can read up on the functionality of numerapi [here](https://github.com/uuazed/numerapi). You can use it to download the competition data, view other numerai users' public profiles, check submission status, manage your stake, and much more. In this case, we'll only be using it to download competition data and submit predictions.

## 1.3 Downloading Competition Data
In a more structured project, you'll probably want to keep the data in a seprate directory from your scripts etc. You could also link google colab to your google drive and store the data there in order to avoid needing to download and process the data every time. In this case, however, we'll keep everything in `./content`, and download the data fresh each time.

In [3]:
napi.download_current_dataset()

./numerai_dataset_254.zip: 100%|█████████▉| 394M/394M [00:11<00:00, 41.4MB/s]2021-03-13 13:40:19,010 INFO numerapi.base_api: unzipping file...
./numerai_dataset_254.zip: 394MB [00:29, 41.4MB/s]                           

'./numerai_dataset_254.zip'

## 1.4 Generating the Training Sample

If you look at the files we downloaded above, you'll see a `numerai_tournament_data.csv` file and a `numerai_training_data.csv` file. The "tournament" file contains many rows with targets which we can use for validation, so let's extract those and combine them with our training set. Note that this cell saves a new `csv` after combining the training and validation data, so we can avoid the time-consuming parsing process if we run this cell again in the same session.

In [4]:
tourn_file = Path(f'./numerai_dataset_{napi.get_current_round()}/numerai_tournament_data.csv')
train_file = Path(f'./numerai_dataset_{napi.get_current_round()}/numerai_training_data.csv')
processed_train_file = Path('./training_processed.csv')

if processed_train_file.exists():
    print("Loading the processed training data from file\n")
    training_data = pd.read_csv(processed_train_file)
else:
    tourn_iter_csv = pd.read_csv(tourn_file, iterator=True, chunksize=1e6)
    val_df = pd.concat([chunk[chunk['data_type'] == 'validation'] for chunk in tqdm(tourn_iter_csv)])
    tourn_iter_csv.close()
    training_data = pd.read_csv(train_file)
    training_data = pd.concat([training_data, val_df])
    training_data.reset_index(drop=True, inplace=True)
    print("Training Dataset Generated! Saving to file ...")
    training_data.to_csv(processed_train_file, index=False)


feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]
target_cols = ['target']

train_idx = training_data.index[training_data.data_type=='train'].tolist()
val_idx = training_data.index[training_data.data_type=='validation'].tolist()


0it [00:00, ?it/s]
1it [00:37, 37.26s/it]
2it [00:59, 30.00s/it]


Training Dataset Generated! Saving to file ...


# 2 Modeling the Data

In this section, we will define our evaluation metrics; run two different models (a linear regression model from `scikit-learn` and a neural network from `fastai`); and generate submission dataframes from those files.

## 2.1 Evaluation Metrics

In this section, we will define two key evaluation metrics used to assess the performance of models before submitting to the tournament. These metrics are:
- Average Spearman Correlation per era: The sum of each era's Spearman correlation divided by the number of eras.
- Sharpe Ratio: The average correlation per era divided by the standard deviation of the correlations per era.

Both are defined in reasonable detail [here](https://wandb.ai/carlolepelaars/numerai_tutorial/reports/How-to-get-Started-With-Numerai--VmlldzoxODU0NTQ). The methods defined below are modified versions of the methods described in that post.

In [5]:
def corr(df: pd.DataFrame) -> np.float32:
    """
    Calculate the correlation by using grouped per-era data
    :param df: A Pandas DataFrame containing the columns "era", "target" and "prediction"
    :return: The average per-era correlations.
    """
    def _score(sub_df: pd.DataFrame) -> np.float32:
        """ Calculate Spearman correlation for Pandas' apply method """
        return spearmanr(sub_df["target"],  sub_df["prediction"])[0]
    corrs = df.groupby("era").apply(_score)
    return corrs.mean() 

def sharpe(df: pd.DataFrame) -> np.float32:
    """
    Calculate the Sharpe ratio by using grouped per-era data
    :param df: A Pandas DataFrame containing the columns "era", "target" and "prediction"
    :return: The Sharpe ratio for your predictions.
    """
    def _score(sub_df: pd.DataFrame) -> np.float32:
        """ Calculate Spearman correlation for Pandas' apply method """
        return spearmanr(sub_df["target"],  sub_df["prediction"])[0]
    corrs = df.groupby("era").apply(_score)
    return corrs.mean() / corrs.std()

## 2.2 Cross Validation Setup

In [ ]:
%%script false --no-raise-error
from sklearn.model_selection import GroupKFold
train_sample = training_data.iloc[train_idx]
X, y, era = train_sample[feature_cols], train_sample.target, train_sample.era
groups = train_sample.era
group_kfold = GroupKFold(n_splits = 5)

In [ ]:
%%script false --no-raise-error
# ALTERNATE SETUP
from sklearn.model_selection import GroupKFold
train_sample = training_data
X, y, era = train_sample[feature_cols], train_sample.target, train_sample.era
groups = train_sample.era
group_kfold = GroupKFold(n_splits = 5)

### 2.2.1 Custom Cross Validation Setup

The goal of this section is to set up a "group time series" approach where we specify a certain set of "eras" for training with the last era for validation. We will be training on segments of the validation set.

There are a few ways to do this; I want to write a class that can take the "eras to test on" as input and return CV folds as outout. Perhaps a future refinement would include an argument for number of eras validate on. Perhaps unnecessary given that the "real" task is testing on a single era. Or four eras?

#### Usage

1. Initialize the class with the eras column: `cv = EraCV(training_data.era)`
2. Get splits: `X, y = test.get_splits(valid_start = 80, valid_n_eras = 4, train_n_eras = None)`

The `valid_start` argument identifies the first training era; it takes an integer value. `valid_n_eras` is the number of eras to include in the validation set. `train_n_eras` is the number of eras to include in the training set. `train_n_eras` before `valid_start` are included in the training set. If no argument is passed to `train_n_eras`, all eras from 0 to `valid_start` are included.

A single instance of this class can be used in a loop to generate multiple train/test splits. Assuming you want to keep the number of train and test eras constant, you can just iterate over a list of validation starting eras.

Features such as checking if a given validation era actually exists have not yet been implemented.

In [135]:
class EraCV:
    """Select validation eras and train on previous eras

    provides train/test indices to split data in train/test splits. In
    each split, one or more eras are used as a validation set while the
    specified number of immediately preceding eras are used as a
    training set.
    """

    def __init__(self, eras):
        self.eras = eras
        self.unique_eras = _era_to_int(eras.unique())
        self.eras_int = _era_to_int(eras)
        #self.valid_start = valid_start
        #self.valid_n_eras = valid_n_eras
        #self.train_n_eras = 0 if (train_n_eras is None) else train_n_eras
    
    def _era_to_int(eras):
        return np.array([int(era[3:]) for era in eras])

    def get_valid_indices(self, valid_start, valid_n_eras):
        self.valid_eras = self.unique_eras[self.unique_eras.index(valid_start):\
                                      self.unique_eras.index(valid_start)+\
                                      valid_n_eras]
        valid_bool = [era in self.valid_eras for era in self.eras_int] 
        self.valid_indices = np.where(valid_bool)

    def get_train_indices(self, valid_start:int, train_n_eras:int):
        train_n_eras = 0 if (train_n_eras is None) else train_n_eras
        self.train_eras = [era for era in self.unique_eras if era <\
                           valid_start][-train_n_eras:]
        train_bool = [era in self.train_eras for era in self.eras_int]
        self.train_indices = np.where(train_bool)

    def get_splits(self, valid_start:int, valid_n_eras:int,
                   train_n_eras:int = None):
        self.get_valid_indices(valid_start, valid_n_eras)
        self.get_train_indices(valid_start, train_n_eras)
        return self.train_indices[0], self.valid_indices[0]

## 2.3 Linear Regression Model
This model closely follows the tutorial example [here](https://colab.research.google.com/github/numerai/example-scripts/blob/master/making-your-first-submission-on-numerai.ipynb). We will use the `scikit-learn` package, with which we can implement and fit our regression model in just a couple of lines of code.

#### Fitting the Linear Regression Model

In [ ]:
%%time
corrs = []
sharpes = []
era_split = EraCV(eras = training_data.era)
X, y, era = training_data[feature_cols], training_data.target, training_data.era
for valid_era in [119, 130, 197]:
    train, test = era_split.get_splits(valid_start = valid_era,
                           valid_n_eras = 4,
                           train_n_eras = None)
    model = sklearn.linear_model.LinearRegression(n_jobs = -1)
    model.fit(X.iloc[train], y.iloc[train])
    val_preds = model.predict(X.iloc[test])
    eval_df = pd.DataFrame({'prediction':val_preds,
                        'target':y.iloc[test],
                        'era':era.iloc[test]}).reset_index()
    corrs.append(corr(eval_df))
    sharpes.append(sharpe(eval_df))

print(corrs)
print(sharpes)

In [ ]:
models = [
          sklearn.linear_model.LinearRegression(n_jobs = -1),
          # sklearn.linear_model.Lasso(alpha=0.00006), # good! Takes pretty long time.
          # sklearn.linear_model.Lasso(alpha=0.00001), # Takes a long time, worse than .00005
          # sklearn.linear_model.Lasso(alpha=0.000005), # Takes a really long time, worse than .00001
          # sklearn.linear_model.Lasso(alpha=0.01), # fails
          sklearn.linear_model.Ridge(alpha=0.0001),
          # sklearn.linear_model.Ridge(alpha=0.01),
          # sklearn.linear_model.Ridge(alpha = 0.1),
          sklearn.linear_model.ElasticNet(alpha=.0001, l1_ratio=0.5),  # Good!
          # sklearn.linear_model.ElasticNet(alpha=.0002, l1_ratio=0.5),
          # sklearn.linear_model.ElasticNet(alpha=.0001, l1_ratio=0.4),
          # sklearn.linear_model.ElasticNet(alpha=.0001, l1_ratio=0.6), # Good! Best?
          # sklearn.linear_model.ElasticNet(alpha=.0001, l1_ratio=0.7) # Good! Best?

          # sklearn.linear_model.ElasticNet(alpha=.00005, l1_ratio=0.5),
          # sklearn.linear_model.ElasticNet(alpha=.0005, l1_ratio=0.5),
          # sklearn.linear_model.ElasticNet(alpha=.00001, l1_ratio=0.25) # Takes very long time
          # sklearn.linear_model.ElasticNet(alpha=.01, l1_ratio=0.5), # fails
          # sklearn.linear_model.ElasticNet(alpha=.001, l1_ratio=0.25),
          # sklearn.linear_model.ElasticNet(alpha=.001, l1_ratio=0.75) # bad result
          ]

In [ ]:
for model in models:
    corrs = []
    sharpes = []
    for train, test in tqdm(group_kfold.split(X, y, groups=groups), total=5):
        model.fit(X.iloc[train], y.iloc[train])
        val_preds = model.predict(X.iloc[test])
        eval_df = pd.DataFrame({'prediction':val_preds,
                            'target':y.iloc[test],
                            'era':groups.iloc[test]}).reset_index()
        corrs.append(corr(eval_df))
        sharpes.append(sharpe(eval_df))
    print(f'\nmodel: {model.__class__.__name__}')
    if model.__class__.__name__!="LinearRegression":
        print(f'alpha: {model.alpha}')
    if model.__class__.__name__=="ElasticNet":
        print(f'L1 Ratio: {model.l1_ratio}')
    print(f'validation correlations: {corrs}, mean: {np.array(corrs).mean()}')
    print(f'validation sharpes: {sharpes}, mean: {np.array(sharpes).mean()}')


100%|██████████| 5/5 [01:12<00:00, 14.40s/it]

  0%|          | 0/5 [00:00<?, ?it/s]


model: LinearRegression
validation correlations: [0.030592672832642553, 0.029802794658032174, 0.03535880547271921, 0.033360716209074485, 0.03270394557743098], mean: 0.032363786949979885
validation sharpes: [1.114464375154921, 0.9485257250293379, 0.9992376387806917, 1.24032712933418, 0.7823271632464406], mean: 1.0169764063091145



100%|██████████| 5/5 [00:13<00:00,  2.79s/it]

  0%|          | 0/5 [00:00<?, ?it/s]


model: Ridge
alpha: 0.0001
validation correlations: [0.030592688376691327, 0.029802788108775426, 0.03535880547271921, 0.03336072298313674, 0.032703961724224004], mean: 0.03236379333310935
validation sharpes: [1.1144649516401044, 0.9485252354835974, 0.9992376387806917, 1.2403277698874937, 0.7823276682925802], mean: 1.0169766528168933



100%|██████████| 5/5 [01:37<00:00, 19.43s/it]


model: ElasticNet
alpha: 0.0001
L1 Ratio: 0.5
validation correlations: [0.0321315350849346, 0.030686260652279727, 0.03937208763992656, 0.03557058382127742, 0.03677966987267029], mean: 0.03490802741421772
validation sharpes: [1.0837542294691465, 0.8349751370328967, 0.9651805955650611, 1.1853246741830534, 0.7697754775344541], mean: 0.9678020227569224


In [ ]:
%%script false --no-raise-error
# Takes a very long time
import sklearn.neighbors
models = [        
          sklearn.neighbors.KNeighborsRegressor(n_neighbors = 500, n_jobs = -1),
          sklearn.neighbors.KNeighborsRegressor(n_neighbors = 500, leaf_size = 150, n_jobs = -1),
          sklearn.neighbors.KNeighborsRegressor(n_neighbors = 500, leaf_size = 10, n_jobs = -1)
                    ]

for model in models:
    corrs = []
    sharpes = []
    for train, test in tqdm(group_kfold.split(X, y, groups=groups), total=5):
        model.fit(X.iloc[train], y.iloc[train])
        val_preds = model.predict(X.iloc[test])
        eval_df = pd.DataFrame({'prediction':val_preds,
                            'target':y.iloc[test],
                            'era':groups.iloc[test]}).reset_index()
        corrs.append(corr(eval_df))
        sharpes.append(sharpe(eval_df))
    print(f'model: {model.__class__.__name__}')
    print(f'neighbors: {model.n_neighbors}')
    print(f'leaf size: {model.leaf_size}')
    print(f'validation correlations: {corrs}, mean: {np.array(corrs).mean()}')
    print(f'validation sharpes: {sharpes}, mean: {np.array(sharpes).mean()}\n')

In [ ]:
%%script false --no-raise-error
# Takes a very long time
import sklearn.ensemble
models = [
          sklearn.ensemble.ExtraTreesRegressor(n_estimators=5, max_features = 0.8, n_jobs=-1),
          sklearn.ensemble.ExtraTreesRegressor(n_estimators=10, max_features = 0.8, n_jobs=-1),
          sklearn.ensemble.ExtraTreesRegressor(n_estimators=5, max_features = 0.2, n_jobs=-1),
          sklearn.ensemble.ExtraTreesRegressor(n_estimators=10, max_features = 0.2, n_jobs=-1),
          # sklearn.ensemble.ExtraTreesRegressor(n_estimators=100, max_features = 0.8, n_jobs=-1),
          # sklearn.ensemble.ExtraTreesRegressor(n_estimators=1000, max_features = 0.8, n_jobs=-1)
                    ]

for model in models:
    corrs = []
    sharpes = []
    for train, test in tqdm(group_kfold.split(X, y, groups=groups), total=5):
        model.fit(X.iloc[train], y.iloc[train])
        val_preds = model.predict(X.iloc[test])
        eval_df = pd.DataFrame({'prediction':val_preds,
                            'target':y.iloc[test],
                            'era':groups.iloc[test]}).reset_index()
        corrs.append(corr(eval_df))
        sharpes.append(sharpe(eval_df))
    print(f'model: {model.__class__.__name__} with {model.n_estimators} estimators and {model.max_features} max features.')
    print(f'validation correlations: {corrs}, mean: {np.array(corrs).mean()}')
    print(f'validation sharpes: {sharpes}, mean: {np.array(sharpes).mean()}\n')

In [ ]:
%%script false --no-raise-error

import sklearn.ensemble
models = [
          sklearn.ensemble.GradientBoostingRegressor(n_estimators=50, max_features = 0.8, subsample=0.1),
          sklearn.ensemble.GradientBoostingRegressor(n_estimators=100, max_features = 0.8),
          sklearn.ensemble.GradientBoostingRegressor(n_estimators=50, max_features = 0.2),
          sklearn.ensemble.GradientBoostingRegressor(n_estimators=100, max_features = 0.2)
                    ]

for model in models:
    corrs = []
    sharpes = []
    for train, test in tqdm(group_kfold.split(X, y, groups=groups), total=5):
        model.fit(X.iloc[train], y.iloc[train])
        val_preds = model.predict(X.iloc[test])
        eval_df = pd.DataFrame({'prediction':val_preds,
                            'target':y.iloc[test],
                            'era':groups.iloc[test]}).reset_index()
        corrs.append(corr(eval_df))
        sharpes.append(sharpe(eval_df))
    print(f'model: {model.__class__.__name__} with {model.n_estimators} estimators and {model.max_features} max features.')
    print(f'validation correlations: {corrs}, mean: {np.array(corrs).mean()}')
    print(f'validation sharpes: {sharpes}, mean: {np.array(sharpes).mean()}\n')

KeyboardInterrupt: ignored

#### Assessing Regression Model Performance

Here we apply the `corr` and `sharpe` methods defined above to predictions made on the validation sample in order to estimate our model's tournament performance.

In [ ]:
# Fit final model
X, y, era = train_sample[feature_cols], train_sample.target, train_sample.era
model = sklearn.linear_model.ElasticNet(alpha=.0001, l1_ratio=0.6)
model.fit(X, y)
val_sample = training_data.iloc[val_idx]
val_preds = model.predict(val_sample[feature_cols])
eval_df = pd.DataFrame({'prediction':val_preds,
                        'target':val_sample.target,
                        'era':val_sample.era}).reset_index()
val_sharpe = sharpe(eval_df)
val_corr = corr(eval_df)

print((f'The linear regression model\'s validation correlation is {val_corr}. '
       f'Its validation sharpe is {val_sharpe}'))

The linear regression model's validation correlation is 0.016638328482573635. Its validation sharpe is 0.4802300833842811


#### Making Predictions with the Regression Model

In [ ]:
ids = []
preds = []

chunksize = 50000

tourn_iter_csv = pd.read_csv(tourn_file, iterator=True, chunksize=1e6)
for chunk in tourn_iter_csv:
    df = chunk[feature_cols]
    out = model.predict(df)
    ids.extend(chunk["id"])
    preds.extend(out)
tourn_iter_csv.close()

In [ ]:
linear_regression_predictions_df = pd.DataFrame({
    'id':ids,
    'prediction':preds
})
linear_regression_predictions_df.head()

,id,prediction
0,n0003aa52cab36c2,0.481178
1,n000920ed083903f,0.492734
2,n0038e640522c4a6,0.524351
3,n004ac94a87dc54b,0.492209
4,n0052fe97ea0c05f,0.502477


In [ ]:
linear_regression_predictions_df.to_csv("lr_predictions.csv", index=False)

#### Submitting Predictions from the Linear Regression Model

We can use `numerapi` to submit these predictions as follows:

In [ ]:
napi.upload_predictions("lr_predictions.csv", model_id=os.environ.get("NUMERAI_MODEL_ID_REGRESSIONS"))

2021-03-06 18:56:10,116 INFO numerapi.base_api: uploading predictions...


'6e5fcc02-19f6-43eb-8aaf-3061dea74c13'